In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 5000
TRAIN_SIZE = 5000

/Users/vik/.virtualenvs/nnets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from text_data import Opus100DatasetWrapper

class Wrapper(Opus100DatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = 40
    target_length = 40

wrapper = Wrapper(SP_VOCAB_SIZE)
datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
valid = datasets["validation"]

Found cached dataset opus100 (/Users/vik/.cache/huggingface/datasets/opus100/en-es/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
100%|██████████| 3/3 [00:00<00:00, 223.38it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=opus100 --vocab_size=5000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: opus100
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extr

In [4]:
def generate(sequence, pred, target, wrapper):
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(target.cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [18]:
class TransformerNetwork(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, blocks=1):
        super(TransformerNetwork, self).__init__()
        self.transformer = nn.Transformer(hidden_units, attention_heads, blocks, blocks, batch_first=True)
        self.embedding = nn.Embedding(input_units, hidden_units)
        k = math.sqrt(1/hidden_units)
        self.output_embedding = nn.Parameter(torch.rand(hidden_units, input_units) * 2 * k - k)

    def forward(self, x, y):
        embed_x = self.embedding(x)
        embed_y = self.embedding(y)
        output = self.transformer(embed_x, embed_y)
        token_vectors = torch.einsum("bse, eo->bso", output, self.output_embedding)
        return token_vectors

In [23]:
from tqdm.auto import tqdm

# TODO: Profile and improve perf - https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html
model = TransformerNetwork(wrapper.vocab_size, 512, 8, blocks=1).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=wrapper.pad_token)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [24]:
EPOCHS = 100
DISPLAY_BATCHES = 2
OUT_SEQUENCE_LEN = wrapper.y_length
PRINT_VALID = True

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad(set_to_none=True)
        pred = model(sequence.to(DEVICE), prev_target.to(DEVICE))

        # If you use a batch, need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        print(f"Epoch {epoch} train loss: {train_loss / len(train) / BATCH_SIZE}")
        sents = generate(sequence, pred, target, wrapper)
        for sent in sents[:DISPLAY_BATCHES]:
            print(sent)

127it [00:34,  3.65it/s]


Epoch 0 train loss: 0.1652786926491054
¡Es el que me robó! | That's the kid that robbed me. | That's........
¡Al suelo! | Get on the ground! | Get! their.


127it [00:34,  3.73it/s]


Epoch 1 train loss: 0.1297462262858556
¡No me dejéis! | Don't leave me! | Don't me!!
Está bien, y entiendo que pude haber sobre reaccionado un poco. | OK, and I get that I may have overreacted a little. | I, I I haveting and was, been., the good I


127it [00:34,  3.69it/s]


...

Epoch 97 train loss: 0.006187685028334537
Sr. Presidente, la historia recordará su nombre por siempre. | Mr. President, history shall forever remember your name. | Mr. President, history shall forever remember your name. President.. name.........................
¿Quién? | - Who is it? | - Who is it?


127it [00:21,  5.92it/s]


Epoch 98 train loss: 0.006058442616270576
Recibí tu mensaje y estoy listo para la acción. | I got your message, man, and I'm ready to roll. | I' your message, and, and I'm ready to roll.
¡Arqueros! | Archers! | Archers!


127it [00:21,  5.96it/s]

Epoch 99 train loss: 0.00619194845624972
Puede tomar las pastillas con o sin comida. | You can take this medicine with or without food. | You can take this medicine without with or food.
¿Lo llevas puesto? | You wear it? | You wear it?
